In [1]:
### 라이브러리 정의하기
# - 모델 또는 파일 저장 라이브러리
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
import time

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
df = pd.read_csv("D:/1조프로젝트/Voice_Phishing/call_type2.csv")

In [3]:
### 결측치 처리하기
# 결측치가 있는 모든 행 삭제하기
df = df.dropna(how = "any")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269 entries, 0 to 268
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      269 non-null    object
 1   phishing  269 non-null    int64 
 2   type      269 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 6.4+ KB


In [4]:
### 훈련 및 테스트 데이터 읽어들이기
train_data = df[:200]
test_data = df[200:]
train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      200 non-null    object
 1   phishing  200 non-null    int64 
 2   type      200 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 200 to 268
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      69 non-null     object
 1   phishing  69 non-null     int64 
 2   type      69 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.7+ KB


In [5]:
### 데이터 정제하기
# - 중복 데이터의 갯수 확인하기
# - 중복되는 데이터가 있다면 duplicate으로 중복 데이터 처리하기
train_data["text"].nunique()

200

In [6]:
### 중복데이터 제거하기
## inplace = True : 별도의 변수에 넣지 않아도 바로 반영됨
train_data.drop_duplicates(subset=["text"], inplace = True)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13104\974164053.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.drop_duplicates(subset=["text"], inplace = True)


In [7]:
len(train_data)

200

In [8]:
### 데이터 정제하기
# - 한글과 공백을 제외하고 모두 제거하기
train_data["text"] = \
    train_data["text"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
train_data

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13104\3030497016.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["text"] = \


,text,phishing,type
0,여보세요 윤정숙씨 되십니까 네 제가 윤정숙인데요 누구시죠 여기는 역삼 경찰서 입니다...,0,2
1,수고하십니다 뭐 쫌 여쭤보려고여쭤볼려고 전화드렸는데요 네 감사합니다 무엇을 도와드...,0,2
2,아 아까 전주시청에 내가 전화했던 사람인데 그 저기 그 뭐 시 시끄럽다고 저 완산...,0,2
3,때문에 몇 가지 확인 차 연락 드렸는데요 본인께서는 전라도 광주 출신인 세 여성 이...,1,2
4,민원인께서 부정 수급 관련해서 신고를 좀 원하시는데 상담 가능할까요 네 한번 연결...,0,2
...,...,...,...
195,네 수고하십니다 여기 서울중앙지검이고요 저는 첨단범죄 수사부에서 근무하고 있는 박승...,1,2
196,다만 아직까지 피해자라고 증명할 증거가 없으셔서 피해자 입증 조사 도와드리려고 하는...,1,2
197,일차 공판의 증거자료로 활용이 되는데 주변에 소음이 크게 들리거나 제자의 목소리가 ...,1,2
198,만약에 아버지가 전 남편이 아들을 애들을 왜 반찬을 부실하게 줬다면 그게 신고가 들...,0,2


In [9]:
## (for문을 많이 돌려야되므로 데이터프레임에서 처리하기)
### 특정 행에 공백(스페이스 한 칸)이 있는 데이터 조회해서
# NaN(null)으로 변환한 뒤에
# null인 행 삭제 함수를 사용해서 삭제시키기
### 공백이 있는 행을 찾기
train_data["text"] = \
        train_data["text"].str.replace("^ +", "", regex=True)

### 띄어쓰기가 없는 공백들을 모두 NaN 처리하기
train_data["text"].replace("", np.nan, inplace = True)

### 결측치가 있다면, 행 제거하기
train_data = train_data.dropna(how = "any")

### 전체 데이터에서 결측치 확인하기
train_data.isnull().sum()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13104\1933419508.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["text"] = \
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13104\1933419508.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["text"].replace("", np.nan, inplace = True)


text        0
phishing    0
type        0
dtype: int64

In [10]:
### 단어 토큰화
# - 불용어 처리 동시에 진행
okt = Okt()

In [11]:
### 불용어 정의하기
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords

[['휴'],
 ['아이구'],
 ['아이쿠'],
 ['아이고'],
 ['어'],
 ['나'],
 ['우리'],
 ['저희'],
 ['따라'],
 ['의해'],
 ['을'],
 ['를'],
 ['에'],
 ['의'],
 ['가'],
 ['으로'],
 ['로'],
 ['에게'],
 ['뿐이다'],
 ['의거하여'],
 ['근거하여'],
 ['입각하여'],
 ['기준으로'],
 ['예하면'],
 ['예를 들면'],
 ['예를 들자면'],
 ['저'],
 ['소인'],
 ['소생'],
 ['저희'],
 ['지말고'],
 ['하지마'],
 ['하지마라'],
 ['다른'],
 ['물론'],
 ['또한'],
 ['그리고'],
 ['비길수 없다'],
 ['해서는 안된다'],
 ['뿐만 아니라'],
 ['만이 아니다'],
 ['만은 아니다'],
 ['막론하고'],
 ['관계없이'],
 ['그치지 않다'],
 ['그러나'],
 ['그런데'],
 ['하지만'],
 ['든간에'],
 ['논하지 않다'],
 ['따지지 않다'],
 ['설사'],
 ['비록'],
 ['더라도'],
 ['아니면'],
 ['만 못하다'],
 ['하는 편이 낫다'],
 ['불문하고'],
 ['향하여'],
 ['향해서'],
 ['향하다'],
 ['쪽으로'],
 ['틈타'],
 ['이용하여'],
 ['타다'],
 ['오르다'],
 ['제외하고'],
 ['이 외에'],
 ['이 밖에'],
 ['하여야'],
 ['비로소'],
 ['한다면 몰라도'],
 ['외에도'],
 ['이곳'],
 ['여기'],
 ['부터'],
 ['기점으로'],
 ['따라서'],
 ['할 생각이다'],
 ['하려고하다'],
 ['이리하여'],
 ['그리하여'],
 ['그렇게 함으로써'],
 ['하지만'],
 ['일때'],
 ['할때'],
 ['앞에서'],
 ['중에서'],
 ['보는데서'],
 ['으로써'],
 ['로써'],
 ['까지'],
 ['해야한다'],
 ['일것이다'],
 ['반드시'],
 ['할줄알다'],
 ['할수있다'],
 [

In [12]:
### 단어 토큰화
# - 훈련데이터
X_train = []
for sentence in tqdm(train_data["text"]) :
    # print(sentence)
    # 토큰화
    tokenized_sentence = okt.morphs(sentence, stem = True)
    # 불용어 처리
    stopwords_removed_sentence = [word
                                     for word in tokenized_sentence
                                         if not word in stopwords]
    # 각 문장별 단어 토큰화 결과 리스트에 추가하기
    X_train.append(stopwords_removed_sentence)
    
print(X_train)

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:23<00:00,  8.69it/s]

[['여보세요', '윤정숙', '씨', '되다', '네', '제', '가', '윤정', '숙이다', '누구', '시', '죠', '여기다', '역삼', '경찰서', '이다', '오늘', '출석요구서', '받다', '고소', '사건', '으로', '조사', '하다', '있다', '내일', '이라도', '당장', '출석', '해주다', '그렇게', '급하다', '내', '가', '무슨', '명예훼손', '을', '하다', '그거', '야', '조사', '를', '해보다', '알다', '빨리', '처리', '하다', '내일', '시', '까지', '출석', '해주다'], ['수고', '하다', '뭐', '쫌', '여쭈다', '보다', '여쭈다', '보다', '전화', '드리다', '네', '감사하다', '무엇', '을', '도', '와', '드릴', '끄다', '네', '제', '말', '좀', '들다', '보다', '이', '하다', '년일', '년', '개월', '십', '개월', '전', '에', '세상', '을', '뜨다', '그', '라고', '저', '그', '라고', '저', '그', '뭐', '뇌출혈', '떨다', '병원', '에서', '하다', '년삼', '년', '고생', '하다', '세상', '을', '뜨다', '이', '병원', '에', '눕다', '사람', '을', '어떻다', '데리', '고', '오다', '인감', '을', '떼다', '여보세요', '인감', '을', '빼다', '이', '메리', '찌다', '화재', '라고', '메리', '찌', '캐피탈', '이라는', '데', '서', '그', '차', '를', '살다', '그', '걸', '로', '하다', '돈', '을', '하다', '이천', '삼백', '몇', '원만', '원', '받다', '더', '라고', '그렇다', '차', '를', '사다', '우리', '동생', '앞', '으로', '병원', '에', '눕다', '있다', '사람', '반신불수', '있다', '사람', 

In [13]:
### 테스트 데이터 정제 작업 먼저 진행
# - 한글과 공백을 제외하고 모두 제거하기
test_data["text"] = \
    test_data["text"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
test_data

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13104\3935041760.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["text"] = \


,text,phishing,type
200,여보세요 여보세요 씨 혹시 맞습니까 네 저는 서울 중앙 지방 검찰청이구요 신인철 수...,1,2
201,이렇게 제가 사실 꿈에서 그랬는지 안 그랬는지 그게 헛갈렸는데 막 이렇게 뛰어오는데...,0,2
202,여보세요 본인 맞으십니까 서울중앙지검 금융범죄 층에 사람입니다 그 연료탱크 성범죄사...,1,2
203,전라도 광주 태생으로 세가 되는 남성이고 이름은 김형석인데 처음 들어보셨습니까 네...,1,2
204,본인 본인의 신상정보를 도용해서 사용하고 있던 김형석이란 사람이 그 사람이 구속이 ...,1,2
...,...,...,...
264,여보세요 예 홍길동 시죠네 그렇습니다 여기 한국 경찰서 인데요 안에 홍길동씨 일전에...,0,2
265,시설에 관련해서 문의주신 선생님 맞으십니까 확인 중에 통화가 종료되어서 부득이하게...,0,2
266,내 사랑 드렸습니다 본인 성함 말씀 하세요 이따 공장 받지 못 하시면 지금 보낸 같...,1,2
267,합법적으로 제 거 개설한거 통장 말씀을 해드릴게요 계좌 번호말고요 농협에 청약에 삼...,1,2


In [14]:
### 특정 행에 공백(스페이스 한 칸)이 있는 데이터 조회해서
# NaN(null)으로 변환한 뒤에
# null인 행 삭제 함수를 사용해서 삭제시키기
### 공백이 있는 행을 찾기
test_data["text"] = \
        test_data["text"].str.replace("^ +", "", regex=True)

### 띄어쓰기가 없는 공백들을 모두 NaN 처리하기
test_data["text"].replace("", np.nan, inplace=True)

### 결측치가 있다면, 행 제거하기
test_data = test_data.dropna(how = "any")

### 전체 데이터에서 결측치 확인하기
test_data.isnull().sum()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13104\2935756037.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["text"] = \
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13104\2935756037.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["text"].replace("", np.nan, inplace=True)


text        0
phishing    0
type        0
dtype: int64

In [15]:
### 단어 토큰화
# - 테스트 데이터
## 공백이 있으면 안되므로 확인하기
X_test = []
for sentence in tqdm(test_data["text"]) :
    # 토큰화
    tokenized_sentence = okt.morphs(sentence, stem = True)
    # 불용어 처리
    stopwords_removed_sentence = [word
                                     for word in tokenized_sentence
                                         if not word in stopwords]
    # 각 문장별 단어 토큰화 결과 리스트에 추가하기
    X_test.append(stopwords_removed_sentence)
    
print(X_test)

100%|██████████████████████████████████████████████████████████████████████████████████| 69/69 [00:06<00:00, 10.35it/s]

[['여보세요', '여보세요', '씨', '혹시', '맞다', '네', '저', '는', '서울', '중앙', '지방', '검찰청', '이구', '요', '신인철', '수', '사관', '이다', '피해자', '네네', '네', '다르다', '아니다', '본인', '앞', '으로', '연결', '되다', '사건', '때문', '에', '몇', '가지', '확인', '하다', '위해', '서', '전화', '드리다', '통화', '는', '괜찮다', '네', '본인', '앞', '으로', '연결', '되다', '사건', '때문', '에', '몇', '가지', '확인', '차', '전화', '드리다', '거', '십', '니', '다', '통화', '는', '괜찮다', '네', '네', '혹시', '서울', '출신', '이고', '삼십칠', '세', '여성', '최민경', '이라고', '알다', '아니다', '모르다', '처음', '들다', '보다', '네', '제', '가', '여쭈다', '보다', '이유', '는', '얼마', '전', '에', '저희', '지', '검', '에서', '최민경', '을', '주', '범', '으로', '되다', '금융', '사', '기단', '을', '검거', '하다', '누가', '현장', '에서', '본인', '명의', '로', '제일', '은행', '하다', '대신', '증권', '두', '계좌', '가', '있다', '전화', '드리다', '네', '네', '보다', '계좌', '개설', '에', '데', '하다', '아시', '는', '내용', '이', '있다', '아니다', '전혀', '모르다', '있다', '말씀', '이세', '요', '네', '네', '일단', '저희', '쪽', '에서', '각', '은행권', '에', '확인', '하다', '보다', '결과', '이천', '십', '칠', '년', '십이월', '달', '에', '서울', '서초구', '쪽', '에서', '새롭다', '개설', '되다', '다',

### 정수 인코딩 하기

In [16]:
### 정수 인코딩을 위한 객체 생성
tokenizer = Tokenizer()

In [17]:
### 정수 인코딩하기 : 딕셔너리 형태로 만들어짐
# - 빈도를 자동으로 계산해서 빈도가 높은 순서대로 1부터 값을 부여시킴
tokenizer.fit_on_texts(X_train)
print(tokenizer.word_index)

{'하다': 1, '이': 2, '네': 3, '가': 4, '에': 5, '을': 6, '되다': 7, '있다': 8, '본인': 9, '를': 10, '그': 11, '거': 12, '예': 13, '뭐': 14, '으로': 15, '아니다': 16, '제': 17, '지금': 18, '그렇다': 19, '은': 20, '아': 21, '요': 22, '는': 23, '에서': 24, '없다': 25, '저희': 26, '드리다': 27, '통장': 28, '로': 29, '말씀': 30, '도': 31, '이제': 32, '확인': 33, '좀': 34, '받다': 35, '사건': 36, '안': 37, '보다': 38, '적': 39, '수': 40, '서': 41, '게': 42, '부분': 43, '고': 44, '들': 45, '계좌': 46, '사': 47, '알다': 48, '때문': 49, '예요': 50, '만': 51, '은행': 52, '돼다': 53, '저': 54, '어떻다': 55, '조사': 56, '피해자': 57, '분': 58, '일단': 59, '의': 60, '같다': 61, '하고': 62, '혹시': 63, '금융': 64, '기범': 65, '명의': 66, '어': 67, '한': 68, '께서': 69, '모르다': 70, '다': 71, '대해': 72, '진행': 73, '연락': 74, '사용': 75, '전화': 76, '하나': 77, '것': 78, '쪽': 79, '이다': 80, '니까': 81, '씨': 82, '한테': 83, '인': 84, '번호': 85, '해주다': 86, '죠': 87, '이렇다': 88, '내용': 89, '사람': 90, '가지': 91, '맞다': 92, '면': 93, '이렇게': 94, '나오다': 95, '님': 96, '근데': 97, '선생님': 98, '않다': 99, '되어다': 100, '개설': 101, '정도': 102, '그러면': 103, '

### 단어 집합 확인하기

In [18]:
### 희귀단어 기준 정의
threshold = 3

# - 전체 단어 수
total_cnt = len(tokenizer.word_index)

### 희귀단어 기준보다 작은 단어의 개수
rare_cnt = 0

### 훈련 데이터의 전체 단어 빈도수 총합
total_freq = 0

### 희귀단어 기준보다 작은 단어의 빈도수의 총합
rare_freq = 0

In [19]:
### 단어 빈도 개수 확인하기 : 리스트의 튜플 형태로 나옴
tokenizer.word_counts

OrderedDict([('여보세요', 151),
             ('윤정숙', 1),
             ('씨', 239),
             ('되다', 1851),
             ('네', 2486),
             ('제', 863),
             ('가', 2480),
             ('윤정', 1),
             ('숙이다', 1),
             ('누구', 24),
             ('시', 138),
             ('죠', 217),
             ('여기다', 13),
             ('역삼', 1),
             ('경찰서', 131),
             ('이다', 250),
             ('오늘', 92),
             ('출석요구서', 1),
             ('받다', 493),
             ('고소', 31),
             ('사건', 475),
             ('으로', 904),
             ('조사', 349),
             ('하다', 4182),
             ('있다', 1839),
             ('내일', 43),
             ('이라도', 23),
             ('당장', 6),
             ('출석', 19),
             ('해주다', 218),
             ('그렇게', 110),
             ('급하다', 2),
             ('내', 180),
             ('무슨', 57),
             ('명예훼손', 1),
             ('을', 1977),
             ('그거', 111),
             ('야', 21),
             ('를', 1436),

In [20]:
### 단어와 빈도수의 쌍을 key와 value로 받아서 반복 처리하기
for key, value in tokenizer.word_counts.items() :
    total_freq = total_freq + value
    
    ### 희귀단어 기준보다 작으면
    if value < threshold :
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

In [21]:
print("단어 집합의 크기 : ", total_cnt)
print("희귀 단어 기준보다 작은 단어수 : ", threshold - 1, rare_cnt)
print("단어 집합에서 희귀 단어의 비율 : ", rare_cnt / total_cnt * 100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 : ", rare_freq / total_freq * 100)

단어 집합의 크기 :  4642
희귀 단어 기준보다 작은 단어수 :  2 2363
단어 집합에서 희귀 단어의 비율 :  50.9047824213701
전체 등장 빈도에서 희귀 단어 등장 빈도 :  2.657466928998214


In [22]:
### 정수인코딩에 적용할 단어의 갯수(사이즈) 결정하기
vocab_size = total_cnt - rare_cnt + 1
print("사이즈 : ", vocab_size)

사이즈 :  2280


In [23]:
### 텍스트 시퀀스(문자)를 정수 시퀀스(정수값)으로 변환
tokenizer = Tokenizer(vocab_size)

### 사이즈만큼의 단어들에 대해서 변환하기
tokenizer.fit_on_texts(X_train)

### 훈련 및 테스트 데이터 변환 적용하기
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
print(X_train,"\n")
print(X_test)

[[130, 82, 7, 3, 17, 4, 589, 146, 87, 849, 152, 80, 211, 35, 484, 36, 15, 56, 1, 8, 369, 595, 1348, 667, 86, 184, 108, 4, 299, 6, 1, 182, 634, 56, 10, 176, 48, 425, 119, 1, 369, 146, 114, 667, 86], [436, 1, 14, 1917, 185, 38, 185, 38, 76, 27, 3, 294, 886, 6, 31, 235, 112, 341, 3, 17, 166, 34, 104, 38, 2, 1, 127, 730, 344, 730, 177, 5, 1492, 6, 802, 11, 169, 54, 11, 169, 54, 11, 14, 1684, 426, 24, 1, 127, 1068, 1, 1492, 6, 802, 2, 426, 5, 90, 6, 55, 44, 147, 6, 1148, 130, 6, 709, 2, 1918, 169, 1918, 1685, 261, 282, 41, 11, 121, 10, 648, 11, 218, 29, 1, 180, 6, 1, 803, 987, 178, 107, 35, 213, 169, 19, 121, 10, 686, 215, 1919, 157, 15, 426, 5, 8, 90, 8, 90, 157, 15, 121, 10, 467, 1686, 731, 148, 90, 2, 1920, 635, 1223, 90, 15, 1349, 18, 114, 8, 11, 535, 153, 14, 88, 732, 2, 121, 236, 121, 121, 121, 39, 227, 176, 42, 300, 887, 459, 55, 157, 15, 18, 114, 1349, 1921, 459, 370, 20, 14, 888, 515, 322, 42, 8, 18, 988, 58, 2, 1223, 30, 2, 988, 58, 2, 25, 988, 58, 2, 18, 1, 11, 58, 157, 15, 18, 1

In [24]:
### 훈련 및 테스트 종속변수 생성하기
y_train = np.array(train_data["phishing"])
y_test = np.array(test_data["phishing"])
print(y_train, "\n")
print(y_test)
## 데이터 프레임에 있는 데이터를 배열 형태로 받아옴

[0 0 0 1 0 1 1 1 1 0 1 1 0 1 0 1 0 1 1 0 1 0 0 1 0 1 0 1 1 1 0 1 1 1 1 0 1
 0 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 0 1 1 1 1
 1 1 1 1 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 0 0 1 1 0 0
 0 1 0 0 0 0 1 1 0 1 1 1 0 1 1 1 0 1 0 1 0 0 0 0 0 1 1 0 1 1 1 1 1 0 0 1 1
 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1
 0 1 1 0 0 1 1 0 1 1 1 1 1 0 0] 

[1 0 1 1 1 0 1 1 1 0 1 1 0 1 0 0 1 0 1 1 1 1 0 0 1 0 1 0 0 0 1 1 1 1 0 1 0
 0 1 0 1 0 0 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 0 1 0 1 1 1 0 0 1 1 1]


In [25]:
"""
### X_train 데이터에 비어있는 데이터 찾아서 삭제하기
# - 길이가 0인 인덱스 삭제하는 방법으로...

### 각 리스트의 길이가 0인 인덱스 찾기
drop_train = [index for index, sentence in enumerate(X_train)
                          if len(sentence) < 1]

### 찾아낸 인덱스 삭제하기
# - 훈련과 독립변수 및 종속변수 모두 동일한 위치의 인덱스 삭제
X_train = np.delete(X_train, drop_train, axis = 0)
y_train = np.delete(y_train, drop_train, axis = 0)

print(len(X_train), len(y_train))
"""

'\n### X_train 데이터에 비어있는 데이터 찾아서 삭제하기\n# - 길이가 0인 인덱스 삭제하는 방법으로...\n\n### 각 리스트의 길이가 0인 인덱스 찾기\ndrop_train = [index for index, sentence in enumerate(X_train)\n                          if len(sentence) < 1]\n\n### 찾아낸 인덱스 삭제하기\n# - 훈련과 독립변수 및 종속변수 모두 동일한 위치의 인덱스 삭제\nX_train = np.delete(X_train, drop_train, axis = 0)\ny_train = np.delete(y_train, drop_train, axis = 0)\n\nprint(len(X_train), len(y_train))\n'

In [26]:
### 패딩 크기 결정하기
count = 0
max_len = 800
for sentence in X_train :
    if(len(sentence) <= max_len) :
        count = count + 1
print("전체 데이터 중에 길이가 {} 이하인 비율 : {} %".format(
            max_len, (count/len(X_train)) * 100))

전체 데이터 중에 길이가 800 이하인 비율 : 91.5 %


In [27]:
### 모든 단어의 길이를 max_len으로 맞추기
# - 패딩 처리
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

### LSTM 훈련시키기

In [28]:
### 라이브러리 정의하기
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [29]:
### LSTM 모델 생성 및 계층 추가하기
embedding_dim = 100
hidden_units = 128

model = Sequential()

model.add(Embedding(6392, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation="sigmoid"))

In [30]:
### 콜백 함수 정의
es = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=3)
mc = ModelCheckpoint("./type2_best_model.h5", monitor="val_acc", mode="max",
                     verbose=1, save_best_only=True)

In [31]:
### 모델 속성 정의하기 : compile
model.compile(optimizer = "adam",
              loss="binary_crossentropy",
              metrics=["acc"])

In [32]:
### 모델 훈련시키기
history = model.fit(X_train, y_train, epochs=50,
                    callbacks=[es, mc], batch_size=64,
                    validation_split=0.2)

Epoch 1/50
3/3 [==============================] - ETA: 0s - loss: 0.6901 - acc: 0.6125
Epoch 1: val_acc improved from -inf to 0.72500, saving model to .\type2_best_model.h5
3/3 [==============================] - 8s 2s/step - loss: 0.6901 - acc: 0.6125 - val_loss: 0.6738 - val_acc: 0.7250
Epoch 2/50
3/3 [==============================] - ETA: 0s - loss: 0.6739 - acc: 0.6438
Epoch 2: val_acc did not improve from 0.72500
3/3 [==============================] - 2s 783ms/step - loss: 0.6739 - acc: 0.6438 - val_loss: 0.6437 - val_acc: 0.7250
Epoch 3/50
3/3 [==============================] - ETA: 0s - loss: 0.6450 - acc: 0.6438
Epoch 3: val_acc did not improve from 0.72500
3/3 [==============================] - 2s 748ms/step - loss: 0.6450 - acc: 0.6438 - val_loss: 0.5760 - val_acc: 0.7250
Epoch 4/50
3/3 [==============================] - ETA: 0s - loss: 0.6726 - acc: 0.6438
Epoch 4: val_acc did not improve from 0.72500
3/3 [==============================] - 2s 757ms/step - loss: 0.6726 - acc:

In [33]:
### 테스트하기
model.evaluate(X_test, y_test)

3/3 [==============================] - 0s 94ms/step - loss: 0.6022 - acc: 0.7536


[0.6022422313690186, 0.7536231875419617]

In [34]:
### 모델 불러들이기
### 훈련 과정에서 검증 데이터의 정확도가 가장 높았을 때
#   저장된 모델인 "naver_best_model.h5"를 로드
loaded_model = load_model("type2_best_model.h5")

### 감성 분류하기

In [35]:
def sentiment_predict(new_sentence) :
    ### 한글 및 공백 이외 모두 제거
    new_sentence = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", new_sentence)
    ### 단어 토큰화 : 어간(표제어) 기준
    new_sentence = okt.morphs(new_sentence, stem=True)
    ### 불용어 처리
    new_sentence = [word for word in new_sentence
                           if not word in stopwords]
    ### 정수 인코딩
    encoded = tokenizer.texts_to_sequences([new_sentence])
    ### 패딩 사이즈 처리
    pad_new = pad_sequences(encoded, maxlen = max_len)

    ### 감성 예측(긍정 or 부정)
    score = float(loaded_model.predict(pad_new))

    if (score > 0.5) :
        print("{}% 확률로 기관사칭형 보이스피싱 입니다.".format(score * 100))
    else :
        print("{}% 확률로 기관사칭형 보이스피싱이 아닙니다.".format((1 - score) * 100))

In [36]:
sentiment_predict("경찰서 내일 오실 수 있으세요 아 내일은 좀 어렵고요 그럼 내일모레는 어떠세요 내일모레도 좀 어려울 거 같고 요번 주는 좀 어려울 거 같아요 제가 좀 바빠서요 아 다음 주나 다다음 주에 어떨까요 어금 다다음 주 몇 월 며칠 어떠세요 안에 그날은 괜찮습니다 아 그럼 그날 뵙겠습니다")

50.04217028617859% 확률로 기관사칭형 보이스피싱 입니다.


In [37]:
sentiment_predict("본인 혹시 전라도 광주 출신의 이현주라고 아십니까? 전혀 들어보신 적은 없으시다는 거죠? 네.네. 왜냐하면 저희 수사과에서 이현주로 인한 금융사기단이 검거가 되었는데 이들 검거현장에서 대량의 대포통장과 신용카드가 압수가 되었습니다.네.중요한건 이 압수물품 중에 본인 명의로 되신 하나은행하고 우리은행 통장 두 점이나 같이 발견되었고요. 본인 혹시 이 통장 개설 건에 대해서는 알고 계셨습니까?네. 하나은행과 우리은행 통장 개설에 대해서 전혀 모르고 계시다는 거죠?네.네. 그러시면 최근 3년 사이에 지갑이거나 신분증, 개인 정보가 유출될만한 물건들을 분실하셨다거나 남에게 양도한 사실이 있습니까?아니요. 없습니다.뭐 여권이거나 운전면허증은요?네. 없습니다.아 전혀 분실하신적도 없으시고 이현주씨도 전혀 들어보신 적도 없으시다는 거죠?네.네. 일단 이번 사건에는 본인뿐만 아니라 연루돼있는 수만도 수백명이나 되시는데 그중 일부분은 돈을 받고 통장을 양도 판매한 사실이 있었고요.오늘 저희 검찰청에서 본인 앞으로 연락드린 원인은 본인께서 이 통장을 직접 개설하시고 금전적인 대가를 받고 양도를 하신건지 아니시면 명의가 도용당한 피해자 신분이신지 그 부분에 대해서 확인 차 연락드렸습니다.어. 지금 저희들도 본인 앞으로만 조사를 하는 게 아니라 본인 앞으로 조사를 마치면 또")

62.5130295753479% 확률로 기관사칭형 보이스피싱 입니다.


In [38]:
sentiment_predict("'여자들이 범죄가 변경이 되기 시작합니다', '기능 범죄로 바뀌게 되는데 어차피이라고 제가 얘기해요', '네 삼차 피해 같은 경우 에는 본인의 자산을 계약 인출하는 방법이 아니라 본인의 적금이나 청약 예금 원래 상품을 담보로 신청 후에 대출금을 받아서 은행을 시작하는 거예요', '네 자의 과정 중에 이 차 피 같은 경우 에는 피해자분들이 인터넷 뱅킹이나 모바일 뱅킹을 통해서 사실에 대한 인지가 빨랐다는 거예요', '네 어차피가 조금 입금을 해야 돼서 인출할 사람이라고 얘기를 드렸어요', '네 모바일 뱅킹이나 인터넷 뱅킹 조회를 하게 되면 해당이 저희가 되잖아요', '그쵸 이런 부분 조회했을 때 본인도 모르게 본인 계좌 해약이 되어 있으니까', '이 부분 이야기 생각해서 확인 후에 빠른 대출 가능했다는 거예요', '네 그래서 제가 돈을 검사를 했는데 삼차 피해가 튼 대출 금액 같은 경우 에는 내가 대부분이 모바일 뱅킹이나 인터넷 뱅킹 통해서도 피해 사실에서인지를 못했다는 거죠.', '네 왜냐면 담보를 설정 돼있는 부분이다 보니까', '그래서 제가 해야 되는 게 아니잖아요', '계좌 그대로 존재를 하고 저 예치까지 가능했기 때문에 이런 부분에 대해서인지를 못했다는 겁니다', '네 삼차 피해가 어떤 식으로 접수가 됐냐면 어떤 피해자가 적금이 만기가 된 거요', '네 저희가 많다 된 이후에 실제로 금융권에 해지를 신청하라고 합니다', '네 어 금융권 직원분이 하겠죠.', '그 예를 들어서 1,000만원을 수령 받아야 되는데신다는 말씀을 가져가도 얘기한 거예요', '예를 들어서 금액이 백만원만 처리 가능하냐라고 얘기 했더니 사 백만원은 담보 설정 돼있기 때문에 최종 불가능하다고 얘기한 거예요', '그때서야 피해 사실에 대해서 계속도 동일하게 된 거죠.', '네 지금 통화하고 계시는 본인 통화 천 그이 차 피나 삼차 피해를 보실 수 있는 상황이기 때문에 본인께서 이러한 그 피 대충 부분에 대해서 진행이 되셔야 된다는 거예요', '네 비 적용되시면 금융권 보안 상향 신청 이루어 진다는 겁니다', '네 보안 등급 자체가 상향 조정된단 얘기에요', '네 일단 그럼 본인이 대출 받기 위해서 전화 받으신 내용에 대해 설명하게 설명을 좀 드리도록 하겠습니다', '네 정상 본인 앞으로 금융 감독원에서 넣어 드리는 재산 보험 신청서라는 게 있어요', '네 자이 부분이 뭐냐면 본인이 수중에 제삼자라던지 김재 문의를부터 2차 피해가 삼차 피해 금전적으로 피해 보시는 부분에 대해서는 이제 재산 보험 신청서를 통해서 보상 처리가 가능하다는 겁니다', '아 예 본인께서 이 과정 중에서 김재원 일행과 금전적인 거래가 있다.든지 이런 부분이 발견된다면 보상 물론 이고 본인 또한 처벌을 받을 수가 있겠죠..', '이 점 참고해주시구요', '열한 조사는 금일 금융 감독원으로 직접 내 방을 해서 자금에 대한 설명을 진행할 거예요', '제가 직접한다면 네네 그냥 한번 제가 설명을 좀 드릴 거예요', '예 일단 설명 듣고 궁금하신 부분이 있으면 질문해 주시거나 이런 부분에 대해서 자금 증명 부분 진행을 할 텐데 일차적으로 진행을 하신 내용이 있을 거예요', '네 이지철 내용과 감독원측에서 계좌 추적 조회 결과가 동일하게 나와줘야 된다는 겁니다', '나라 동일하게 나와줘야지', '본인이 피해자라는 부분을 법정 받을 수 있단 얘기에요', '그럼 수사관에게 진술하셨던 내용을 한번 진술가 들어갈 텐데 그 본인께서 진행해 주신 내용 자체가 거짓 없는 확인이요', '아 그러면 자료는 제가 금융 감독원 쪽으로 제출하도록 할 거예요', '본인께서 새마을 금고아 신한 은행 두군데 금융권 사용하신 부분이 맞으신 거고 네 그리고 제가 전화 구만원 정도 회복을 하고 싶은 생각에 처리하고 계신 장사하시니까', '정확하게 모르시는 부분이고 저희가 확인을 해보도록 할게요', '생활금고 하시는데 각각 하나의 계좌가 있으신 부분이 맞으세요', '네 최 최소 액티브 고객님 믿고 기다려드린다고', '예 전화 온 거 저희 쪽 계좌에 삼천만원가량 본점 있고 진술하셨는데요', '뭐 본인께서 직접 했던 부분입니까', '지금까지 뭐 계속 그 자금을 얘기를 해주셨던 거예요', '보신 거예요', '알겠습니다', '일단 이 부분 제가 금융 감독원 쪽으로 전화 준다 진행하도록 할께요', '일단 본인이 이번 사건의 피해자가 맞다면 본인의 개인 정보가 언제 어떻게 처리되었고 통장 같은 경우에는 언제 어떻게 한자에게 계산됐는지 2분 현재 수사가 진행 중이에요', '네 그렇기 때문에 금일 피해 직장이 완료되시기 전에 이번 사건 내용에 대해 서어 제 삼자에게 발송이 될 경우에 문제가 될 수 있단 얘기에요', '아 본인 이외에 제 삼자에게 사건 내용이 발생될 경우에 문제가 될 수 있단 얘깁니다', '본인께서 뭐 이런 사건에 연루가 돼서 뭐 검찰 조사를 받고 있다.', '하나 이런 부분으로 진행이 되더라 그냥 내용이 다 발송이 되게 되면 본인께서는 뭐 주변 지인분들이 라든지 가족분들에게 뭐 다친걸 수도 있겠지만 저희 쪽에서 봤을 때는 본인이 정말 주변 지인분들이나 가?', '이해가 되세요', '그렇기 때문에어 지금 본인께서 본인께서 피해자다라는 부분이 입증되시기 전까지를 사건 내용에 대해서 한번 해주셔야 된단 얘기에요', '응 이해가 되시죠.', '본인인 제 구속 수사 대상자가 아닌 건 알고 계시죠.', '네에 앞서 말씀 드렸을 거예요', '일차적으로 수술한 분이 본인께 연락을 취하기 전에 통화를 좀 해봤다고 말씀드렸습니다', '만약에 여기서 본인이 김 재무 드린 결과")

63.89575004577637% 확률로 기관사칭형 보이스피싱 입니다.


In [39]:
sentiment_predict("한국 경찰서 인데요 안에 경찰서 좀 출석 하실 수 있으세요 아 무슨 일인가요 별 일은 아니고요 그냥 가벼운 마음으로 경찰서 오시면 돼요 별일 아닌데 어떻게 경찰서에서 저희 불러요 아 진짜 별일 아니니까 너무 걱정하지 마시고요 경찰서 오시면 됩니다 내일 시간 되세요 안 내일은 시간이 되는데요 아 그럼 내일 오시면 되겠네요 아네 알겠습니다 근데 저 경찰서 가는데 변호사 하고 같이 가도 될까요 아 별일 아닌데 뭐 변호사까지 선임해서 오실 필요가 있으시겠어요 변호사 선임하면 만드는데 그냥 오셔도 돼요 아네 알겠습니다")

55.505406856536865% 확률로 기관사칭형 보이스피싱이 아닙니다.


In [40]:
import speech_recognition as sr
import pyaudio
# 형태소 분석
from konlpy.tag import Okt
import pandas as pd
okt = Okt()
# 인식할 수 없는 형태의 음원 파일을 변환하기 위한 모듈
from pydub import AudioSegment
# wav 파일 길이를 가져오기
import wave
import contextlib

# 생성된 wav 파일을 지우기 위한 모듈
import os

class voice:    
    def __init__(self, input_file):
        self.file = input_file   # 분석을 원하는 음성 파일
        self.r = sr.Recognizer() # sr의 recognizer 불러옴
        
        self.cnt = 1        # 보이스피싱 확률 변수
        self.type1_cnt = 1  # 대출사기형 확률
        self.type2_cnt = 1  # 수사기관사칭형 확률        
        self.text = ''      # 음성에서 변환된 텍스트
        self.export_cnt = 0 # 새롭게 wav 파일이 만들어졌는지 여부를 알기 위함
        
    # 음성 파일을 wav 파일로 통일하는 함수
    def to_wav(self):
        try:
            if self.file[self.file.rfind('.')+1:] != 'wav':
                sound = AudioSegment.from_file(self.file) 
                self.file = self.file[:self.file.rfind('.')]+'.wav'
                sound.export(self.file , format="wav")  # 파일을 인식할 수 있도록 파일 형식 변환
                self.export_cnt = 1    # 새롭게 wav 파일이 만들어짐
            
            with contextlib.closing(wave.open(self.file,'r')) as f:
                frames = f.getnframes()
                rate = f.getframerate()
                duration = frames / float(rate)
                
            self.duration_list = [30]*int(duration/30) + [round(duration%30)]
                
        except:
            print('Error')
            
    # 음성을 텍스트로 변환하는 함수        
    def recognize(self):
        try:
            with sr.AudioFile(self.file) as source:                         # 오디오 파일 불러오기
                for duration in self.duration_list:
                    self.r.adjust_for_ambient_noise(source, duration=0.5)
                    self.r.dynamic_energy_threshold = True
                    audio = self.r.record(source, duration=duration)        
                    try:
                        self.text += self.r.recognize_google(audio_data=audio, language='ko-KR')#집어넣기
                        print('▶ 통화내역 : {}'.format(self.text))  # 통화내역 출력
                    except:
                        None
                
            #print(self.text)
            
            if self.export_cnt == 1:
                if os.path.exists(self.file):
                    os.remove(self.file)               # 텍스트변환 다하면 음성 삭제
                    
        except: 
            print('Error')
    def sentiment_predict(self) :
        ### 한글 및 공백 이외 모두 제거
        self = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", self.text)
        ### 단어 토큰화 : 어간(표제어) 기준
        self = okt.morphs(self, stem=True)
        ### 불용어 처리
        self = [word for word in self
                               if not word in stopwords]
        ### 정수 인코딩
        encoded = tokenizer.texts_to_sequences([self])
        ### 패딩 사이즈 처리
        pad_new = pad_sequences(encoded, maxlen = max_len)

        ### 감성 예측(긍정 or 부정)
        score = float(loaded_model.predict(pad_new))

        if (score > 0.5) :
            print("{}% 확률로 기관사칭형 보이스피싱 입니다.".format(score * 100))
        else :
            print("{}% 확률로 기관사칭형 보이스피싱이 아닙니다.".format((1 - score) * 100))

                        
    # 결과를 출력하는 함수
    def result(self):
        self.to_wav()
        self.recognize() # 음성 텍스트 변환 함수 호출    
        self.sentiment_predict()

In [41]:
from Voice_Phishing import voice_detection as vd
file_path = 'D:/1조프로젝트/Voice_Phishing/test10.wav' # 파일 경로 담기
vp = voice(file_path)
vp.result()

▶ 통화내역 : 지난 여름에 비가 많이 와서 차가 반쯤 잠기는 사고가 있었는데요 시민보험 있더라고요 신청 가능한가요 내 보험 신청 가능합니다 서울시의 물어봐야 돼요 챙겨 물어봐야 돼요 서울시와 계약한 보험 회사에 문의하시면 됩니다 보험 회사가 어디인가요 내일 라이프 생명 보험사입니다 사람이 다친게 아닌 물건에 대한 보상으로 됩니까 시민안전보험은 천재지변에 의한 손에도 보상 하고 있습니다네 제가 개인적으로든 보험
▶ 통화내역 : 지난 여름에 비가 많이 와서 차가 반쯤 잠기는 사고가 있었는데요 시민보험 있더라고요 신청 가능한가요 내 보험 신청 가능합니다 서울시의 물어봐야 돼요 챙겨 물어봐야 돼요 서울시와 계약한 보험 회사에 문의하시면 됩니다 보험 회사가 어디인가요 내일 라이프 생명 보험사입니다 사람이 다친게 아닌 물건에 대한 보상으로 됩니까 시민안전보험은 천재지변에 의한 손에도 보상 하고 있습니다네 제가 개인적으로든 보험있잖아요 내 가능합니다 최대 얼마까지 보상 되는지요 최대 천만 원까지입니다네 알겠습니다네 좋은 하루 되세요
53.05202007293701% 확률로 기관사칭형 보이스피싱 입니다.
